# Deepcamp: Codelab 2
---

**In this tutorial we will cover**:

- Outliers and what to do with them

**Author**:
- Alessio Devoto (alessio.devoto@uniroma1.it)

**Duration**: 30 mins 


# Sales prediction


- You company collected a dataset containing information about investments in commercials via TV, Radio and Newspapers.
- Given the amount of money invested in different advertisement media (TV, Radio, Newspaper) predict sales.

In this notebook we are going to meet **outliers** 😦 for the first time!



What are we going to do? 🤔

1. Data import , analysis & preprocessing ⚙️
2. Train an ML model 
3. Treat outliers
4. Check performance degradation due to outliers

But wait... what actually is an outlier? 

![image](https://raw.githubusercontent.com/alessiodevoto/deepers/main/images/outliers_example.jpg)

From Wikipedia: *in statistics, an outlier is a data point that differs significantly from other observations.*

First, we import the necessary libraries as usual...

In [1]:
!pip install pandas
!pip install scikit-learn
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

... and download the data

In [26]:
!wget https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/Advertising_outliers.csv

--2023-04-21 10:39:25--  https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/Advertising_outliers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4019 (3.9K) [text/plain]
Saving to: ‘Advertising_outliers.csv’

Advertising_outlier 100%[===================>]   3.92K  --.-KB/s    in 0s      

2023-04-21 10:39:25 (52.8 MB/s) - ‘Advertising_outliers.csv’ saved [4019/4019]



## 1. ⬇️ Data import & analysis

In [27]:
# read dataframe from csv
sales_data = pd.read_csv('Advertising_outliers.csv')

In [28]:
sales_data.head()

,TV,Radio,Newspaper,Sales
0,517.0,37.8,69.2,22.1
1,616.0,39.3,45.1,10.4
2,668.0,45.9,69.3,9.3
3,775.0,41.3,58.5,18.5
4,658.0,10.8,58.4,12.9


In [29]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


Let us explore the correlation between each of the three features and the target.

In [30]:
px.scatter(sales_data, x=['TV', 'Radio', 'Newspaper'], y='Sales')

It looks like we have a quite clear ***linear correlation*** between the money spent for TV advertisment and the sales.

Our goal: use the amount of money invested in TV commercial to predict the revenues (i.e. we don't care about Radio and Newspapers for now)

Let's have a look at the distribution of each indipendent variable via a boxplot.

In [31]:
fig = px.box(sales_data, y=["TV", "Radio", 'Newspaper'])
fig.show()

Bad news: there are some ***outliers*** in the TV feature: exactly the one we had chosen for our model! 😞

Well, let's try and train our model anyway, maybe we'll get reasonable performances anyway!

## 2. Train an ML model

Before we start: keep in mind we are training our model on **dirty data** which will probably affect the model's prediction.

### 2.1 Linear Regression

[Linear regression](http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm) is a simple method that looks for the line that best suits the data by minimizing the mean squared error.

In [32]:
# do the usual train test split

sales_data_feat, sales_data_labels = sales_data.drop(columns='Sales'), sales_data['Sales']

X_train, X_test, y_train, y_test = train_test_split(sales_data_feat, sales_data_labels, train_size=0.75)

We can use any of the three columns, but in this case we only keep the TV as we saw there is a strong linear correlation 


In [33]:
# this way we can pick which columns we should use
use_columns = ['TV']

# fit the model
lr = LinearRegression().fit(X=X_train[use_columns].values, y=y_train.values)


Let's see what is the mean error we are getting ...

In [34]:
print('Score:', lr.score(X_test[use_columns].values, y_test))
y_pred = lr.predict(X_test[use_columns].values)
print("mean_absolute_error:    ", metrics.mean_absolute_error(y_test, y_pred))
print("mean_squared_error:    ",metrics.mean_squared_error(y_test, y_pred))
print("squared mean_squared_error:   ",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Score: 0.051688358781960875
mean_absolute_error:     3.674211639715345
mean_squared_error:     23.88383181028013
squared mean_squared_error:    4.887108737308812


Seems to be quite bad 😢, let's try and visualize what's happening.

The result of linear regression is just a line in an N dimesional space, where N=number of features!

We can retrieve the line's equation and plot it!

In [35]:
print('Coefficients: ', lr.coef_)
print('Intercept: ', lr.intercept_)

px.line(x=sales_data['TV'], y=sales_data['TV'] * lr.coef_ + lr.intercept_)


Coefficients:  [0.01424859]
Intercept:  11.499931038727118


Well, that's just a line... 
Now we can visualize how well the line fits the data.

In [36]:
trace1= go.Scatter(mode='markers', x=sales_data['TV'], y=sales_data['Sales'])
trace2 = go.Scatter(x=sales_data['TV'], y=sales_data['TV'] * lr.coef_ + lr.intercept_)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800,xaxis=dict(tickangle=-90))
fig.show()

We can see that the linear model is being misled by the outliers...

![outliers_lr](https://raw.githubusercontent.com/alessiodevoto/deepers/main/images/outliers.jpg)

### Exercise: 🏋 Decision Tree

Perform the same task with a decision tree for regression (`sklearn.tree.DecisionTreeRegressor()`) and plot the results


In [52]:
from sklearn import tree
# this way we can pick which features we should use
use_columns = ['TV']

# fit the model
tree_reg = tree.DecisionTreeRegressor().fit(X=X_train[use_columns].values, y=y_train.values)

In [53]:
print('Score:', tree_reg.score(X_test[use_columns].values, y_test))
y_pred = tree_reg.predict(X_test[use_columns].values)
print("mean_absolute_error:    ", metrics.mean_absolute_error(y_test, y_pred))
print("mean_squared_error:    ",metrics.mean_squared_error(y_test, y_pred))
print("squared mean_squared_error:   ",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Score: 0.17735332272730553
mean_absolute_error:     3.9645833333333336
mean_squared_error:     23.736041666666665
squared mean_squared_error:    4.871964867142072


In [58]:
y_pred = tree_reg.predict(X_test[use_columns].values)
df = pd.DataFrame({'x':X_test[use_columns].values.squeeze(), 'y': y_pred}).sort_values(by='x')
# px.line(df, x='x', y='y')

trace1= go.Scatter(mode='markers', x=sales_data['TV'], y=sales_data['Sales'])
trace2 = go.Scatter(x=df['x'], y=df['y'])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800,xaxis=dict(tickangle=-90))
fig.show()

## 3. Treat outliers & retrain

We can assume outliers are the major responsible for our bad results.

In order to handle the outliers we should first:

1. Find how many outliers we have and where they have  
2. Decide what to do with them 

There are a few [methods](https://regenerativetoday.com/a-complete-guide-for-detecting-and-dealing-with-outliers/) to handle outliers: z-score, Quartiles ...



#### Z-Score 

Let's write a simple function to find out which samples are out of distribution.

We first compute the distribution's mean and standard deviation, and then we check how many 'standard deviations' each point is from the mean.

$ ζ(x) = \frac{(x - μ)}{σ} $

where $\mu$ is the mean and $σ$ is the std deviation.

We can then drop points which are too far from the mean, i.e.



```
if Z_score > threshold:
  drop sample
```


In [46]:

# a simple function to detect outliers
def Zscore_outlier(df: pd.DataFrame, threshold):
    out=[]
    m = np.mean(df) # mean
    sd = np.std(df) # std deviation

    # iterate over the dataset
    for idx, i in enumerate(df): 
        z = (i-m)/sd
        if np.abs(z) > threshold: 
            out.append(idx)
    print("Outliers:", out)
    return out

# apply function to TV column and get indexes of outliers
outliers_idx = Zscore_outlier(sales_data['TV'], threshold=2.5)

Outliers: [1, 2, 3, 4, 5, 6, 7, 8, 9]


In [47]:
# let's just drop the outliers and retrain

sales_data = sales_data.drop(index=outliers_idx)

In [48]:
sales_data_feat, sales_data_labels = sales_data.drop(columns='Sales'), sales_data['Sales']

X_train, X_test, y_train, y_test = train_test_split(sales_data_feat, sales_data_labels, train_size=0.75)

In [49]:
use_columns = ['TV']

# fit the model
lr = LinearRegression().fit(X=X_train[use_columns].values, y=y_train.values)

In [50]:
print('Score:', lr.score(X_test[use_columns].values, y_test))
y_pred = lr.predict(X_test[use_columns].values)
print("mean_absolute_error:    ", metrics.mean_absolute_error(y_test, y_pred))
print("mean_squared_error:    ",metrics.mean_squared_error(y_test, y_pred))
print("squared mean_squared_error:   ",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Score: 0.5936870725376915
mean_absolute_error:     2.6866284930970994
mean_squared_error:     11.723454117536955
squared mean_squared_error:    3.42395299581302


Looks like we got a quite good increase in score! 🍾

---


1. 🚫 Simply ***dropping the outliers*** is not a smart way to deal with them. 
In the vast majority of cases, you should conduct a deeper analysis of the outliers distribution, maybe ask a *domain expert* or replace the outliers with another value.!

2. 🚫 Some ML methods, like Linear Regression, are more sensitive to outliers. Other methods, like decision trees, are more robust. The choice of the ML algorithm you use will affect the robustness of your model!


In [51]:
trace1= go.Scatter(mode='markers', x=sales_data['TV'], y=sales_data['Sales'])
trace2 = go.Scatter(x=sales_data['TV'], y=sales_data['TV'] * lr.coef_ + lr.intercept_)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800,xaxis=dict(
      tickangle=-90
    ))
fig.show()

# Final Exercises 🔥

We probably won't have time but you can do this at home just for fun 😀

## 1. Exercise 1: perform linear regression on the dirty dataset with all the three columns

## 2. Exercise 2: Use SVC instead of linear regression

Hint: the class is called sklearn.svm.SVR


In [55]:
from sklearn import svm

# this way we can pick which features we should use
use_columns = ['TV']

# fit the model
svr = svm.SVR().fit(X=X_train[use_columns].values, y=y_train.values)

In [56]:
print('Score:', svr.score(X_test[use_columns].values, y_test))
y_pred = svr.predict(X_test[use_columns].values)
print("mean_absolute_error:    ", metrics.mean_absolute_error(y_test, y_pred))
print("mean_squared_error:    ",metrics.mean_squared_error(y_test, y_pred))
print("squared mean_squared_error:   ",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Score: 0.5483567622785398
mean_absolute_error:     2.8727361144178247
mean_squared_error:     13.031381521609466
squared mean_squared_error:    3.6099004863859427


In [57]:
y_svr = svr.predict(X_test[use_columns].values)
df = pd.DataFrame({'x':X_test[use_columns].values.squeeze(), 'y': y_svr}).sort_values(by='x')


trace1= go.Scatter(mode='markers', x=sales_data['TV'], y=sales_data['Sales'])
trace2 = go.Scatter(x=df['x'], y=df['y'])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 800,xaxis=dict(tickangle=-90))
fig.show()